[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

# Рецензии на imdb

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Имеются 25000 рецензий пользователей imdb с бинарными метками, посчитанными по оценкам: 0 при оценке < 5 и 1 при оценке >=7.

Полные данные: https://www.kaggle.com/c/word2vec-nlp-tutorial/data

Загрузим выбоку:

In [2]:
path = 'https://raw.githubusercontent.com/chekhovana/courses/main/machine_learning/2_supervised_learning/5_bayes/data/5.7_imdb_reviews.tsv'
imdb = pd.read_csv(path, delimiter='\t')
imdb.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


Классы сбалансированы:

In [ ]:
imdb.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

Разобъём выборку на обучение и контроль:

In [ ]:
from sklearn.model_selection import train_test_split
texts_train, texts_test, y_train, y_test = train_test_split(imdb.review.values, imdb.sentiment.values)

Векторизуем тексты рецензий:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(sublinear_tf=True, use_idf=True)
X_train = vect.fit_transform(texts_train)
X_test = vect.transform(texts_test)

## Логистическая регрессия

Настроим на векторизованных данных логистическую регрессию и посчитаем AUC:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
clf = LogisticRegression()
clf.fit(X_train, y_train)
print(metrics.accuracy_score(y_test, clf.predict(X_test)))
print(metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))

0.8976
0.9635725763481454


Признаков получилось очень много:

In [ ]:
X_train.shape

(18750, 66516)

Попробуем отбирать признаки с помощью лассо:

In [ ]:
clf = LogisticRegression(C=0.15, penalty='l1', solver='liblinear')
clf.fit(X_train, y_train)
print(np.sum(np.abs(clf.coef_) > 1e-4))
print(metrics.accuracy_score(y_test, clf.predict(X_test)))
print(metrics.roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))

104
0.82352
0.9071499474492212


## Метод главных компонент

Сделаем 100 синтетических признаков с помощью метода главных компонент:

In [ ]:
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(n_components=100)
X_train_pca = tsvd.fit_transform(X_train)
X_test_pca = tsvd.transform(X_test)

Обучим на них логистическую регрессию:

In [ ]:
clf = LogisticRegression()
clf.fit(X_train_pca, y_train)
print(metrics.accuracy_score(y_test, clf.predict(X_test_pca)))
print(metrics.roc_auc_score(y_test, clf.predict_proba(X_test_pca)[:, 1]))

0.87056
0.9419559417301316


По 100 полученных таким способом признакам качество получается не намного хуже, чем по всем 66702!

Попробуем обучить на них обучить случайный лес:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train_pca, y_train)
print(metrics.accuracy_score(y_test, clf.predict(X_test_pca)))
print(metrics.roc_auc_score(y_test, clf.predict_proba(X_test_pca)[:, 1]))

0.83264
0.9120012297179237


Признаки, которые даёт метод главных компонент, оптимальны для линейных методов, поэтому логистическая регрессия показывает результаты лучше, чем сложные нелинейные классификаторы.